# Import and Setup

In [1]:
# Imports & setup
from agents import Agent, OpenAIChatCompletionsModel, AsyncOpenAI, Runner, trace
from dotenv import load_dotenv
import requests
import os
import asyncio
import sendgrid
from sendgrid.helpers.mail import Email, To, Content, Mail
from typing import Dict
from IPython.display import display, Markdown
from pydantic import BaseModel, Field
from agents.model_settings import ModelSettings
import markdown  # for converting markdown to HTML

# Load environment variables
load_dotenv(override=True)

# Gemini API setup
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
GEMINI_BASE_URL = os.getenv("GEMINI_BASE_URL")

gemini_client = AsyncOpenAI(
    api_key=GEMINI_API_KEY,
    base_url=GEMINI_BASE_URL
)

model = OpenAIChatCompletionsModel(
    model='gemini-2.0-flash',
    openai_client=gemini_client
)

# Search Agent

In [2]:
# Search tool
def google_search(query: str) -> str:
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")

    url = "https://www.googleapis.com/customsearch/v1"
    params = {"key": GOOGLE_API_KEY, "cx": GOOGLE_CSE_ID, "q": query, "num": 5}

    try:
        resp = requests.get(url, params=params)
        resp.raise_for_status()
        data = resp.json()
        results = []
        for item in data.get("items", []):
            title = item.get("title", "")
            snippet = item.get("snippet", "")
            link = item.get("link", "")
            results.append(f"{title}: {snippet} ({link})")
        return "\n".join(results)
    except Exception as e:
        return f"Google search failed: {e}"

# Search Agent
search_instructions = """
You are a research assistant AI. Given a topic, search and produce a concise, structured summary.

Rules:
- 2-3 paragraphs, under 300 words
- Focus on relevant & recent info
- Short, clear sentences
"""

search_agent = Agent(
    name="Deep Research Agent",
    instructions=search_instructions,
    model=model,
)


# Planner Agent

In [3]:
# Planner Agent
class WebSearchItem(BaseModel):
    reason: str
    query: str

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem]

planner_instructions = "Suggest 5 useful web search queries for the given topic."

planner_agent = Agent(
    name='PlannerAgent',
    instructions=planner_instructions,
    model=model,
    output_type=WebSearchPlan,
)

# Writer Agent

In [4]:
writer_instructions = (
    "Write a comprehensive research report.\n"
    "Requirements:\n"
    "- At least 1500 words\n"
    "- 6–8 sections: Introduction, Background, Analysis, Case Studies, Challenges, Future Directions, Conclusion\n"
    "- Must be in Markdown format\n"
    "- End with 5 follow-up research questions"
)

class ReportData(BaseModel):
    short_summary: str
    markdown_report: str
    follow_up_questions: list[str]

writer_agent = Agent(
    name='Writer Agent',
    instructions=writer_instructions,
    model=model,
    output_type=ReportData
)

# Email Agent

In [5]:
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """Send an email via SendGrid"""
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("deepnagpal147514@gmail.com")  # must be verified in SendGrid
    to_email = To("akagamishanks1554@gmail.com")      # recipient email

    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()

    response = sg.client.mail.send.post(request_body=mail)

    return {
        "status": "success" if response.status_code == 202 else "failed",
        "code": response.status_code,
        "body": response.body.decode() if response.body else "",
    }

# Helper Functions

In [6]:
async def plan_search(query: str):
    result = await Runner.run(planner_agent, f"Query: {query}")
    return result.final_output

async def search(item: WebSearchItem):
    input_text = f"Search term: {item.query}\nReason: {item.reason}"
    result = await Runner.run(search_agent, input_text)
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    return results

async def write_report(query: str, search_results: list[str]):
    input_text = f"Original query: {query}\nSummarized results: {search_results}"
    result = await Runner.run(writer_agent, input_text)
    return result.final_output

async def send_report(report: ReportData, query: str):
    html_body = markdown.markdown(report.markdown_report)
    print("📧 Sending email...")
    response = send_email(f"Research Report: {query}", html_body)
    print("✅ SendGrid Response:", response)
    return response


# Showtime

In [8]:
query = "Future of Artificial Intelligence"  # try also "Future of Artificial Intelligence"

with trace("Research Trace"):
    search_plan = await plan_search(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    email_response = await send_report(report, query)

    display(Markdown(report.markdown_report))  # show in notebook
    print("\n📌 Short Summary:", report.short_summary)
    print("\n📌 Follow-up Questions:", report.follow_up_questions)
    print("\n📌 Email Status:", email_response)

📧 Sending email...
✅ SendGrid Response: {'status': 'success', 'code': 202, 'body': ''}


# The Future of Artificial Intelligence: A Comprehensive Analysis

## 1. Introduction

Artificial Intelligence (AI) has rapidly evolved from a theoretical concept to a tangible force reshaping industries, economies, and societies worldwide. This report delves into the future of AI, exploring key trends, ethical considerations, workforce transformations, potential applications, and the ongoing debate surrounding Artificial General Intelligence (AGI). We will analyze the current state of AI, examine its potential impact on various sectors, and discuss the challenges and opportunities that lie ahead. Furthermore, we will explore case studies that highlight the transformative power of AI, and propose future research directions that can help us better understand and navigate the complex landscape of AI.

## 2. Background: The Rise of AI

The modern era of AI began with the Dartmouth Workshop in 1956, but it wasn't until the advent of powerful computing, massive datasets, and algorithmic breakthroughs like deep learning that AI truly began to flourish. Machine learning, a subset of AI, allows systems to learn from data without explicit programming, enabling them to perform tasks such as image recognition, natural language processing, and predictive analytics. The increasing availability of cloud computing resources has further democratized access to AI, allowing smaller organizations and individuals to leverage its capabilities.

Key milestones in AI development include the creation of expert systems in the 1980s, the victory of IBM's Deep Blue over Garry Kasparov in 1997, and the rise of deep learning in the 2010s. Today, AI is ubiquitous, powering everything from search engines and recommendation systems to autonomous vehicles and medical diagnostics. This rapid progress has fueled both excitement and concern about the future of AI, prompting discussions about its potential benefits and risks.

## 3. Analysis: Key Trends Shaping the Future of AI

Several key trends are shaping the future trajectory of AI:

*   **Democratization of AI:** AI tools and platforms are becoming increasingly accessible, empowering individuals and smaller organizations to leverage AI capabilities without extensive expertise or resources. Automated Machine Learning (AutoML) and cloud-based AI services are democratizing access, fostering innovation across various sectors.
*   **Ethical and Responsible AI:** There is a growing emphasis on ethical and responsible AI development, addressing concerns about bias, transparency, and accountability. This includes the development of frameworks and guidelines to ensure AI systems are aligned with human values and societal norms.
*   **Advancements in NLP:** Natural Language Processing (NLP) continues to evolve, enabling more sophisticated human-computer interactions and improved understanding of textual data. This is driving innovation in areas such as chatbots, machine translation, and sentiment analysis.
*   **Computer Vision Enhancements:** Computer vision is enhancing image and video analysis, with applications ranging from autonomous vehicles to medical diagnosis. Advances in deep learning have significantly improved the accuracy and efficiency of computer vision systems.
*   **AI Convergence with Other Technologies:** The convergence of AI with other technologies like IoT, blockchain, and edge computing is creating new opportunities and driving innovation across industries. This synergy is enabling the development of intelligent systems that can operate in real-time and make autonomous decisions.

## 4. Case Studies: AI in Action

*   **Healthcare:** AI is transforming healthcare through personalized medicine, AI-driven diagnostics, and robotic surgery. AI algorithms can analyze medical images to detect diseases earlier and more accurately, while robotic surgery allows for minimally invasive procedures with greater precision.
*   **Transportation:** Self-driving vehicles, smart traffic management systems, and optimized logistics are revolutionizing transportation. AI-powered autonomous vehicles promise to reduce accidents, improve traffic flow, and enhance mobility for people with disabilities.
*   **Manufacturing:** AI-powered automation and predictive analytics are transforming manufacturing and supply chains, leading to increased productivity and reduced costs. AI algorithms can optimize production processes, predict equipment failures, and improve quality control.
*   **Finance:** AI is being used to detect fraudulent activities, manage risk, and provide personalized financial advice. AI algorithms can analyze large datasets to identify suspicious transactions and assess creditworthiness.
*   **Education:** AI-driven education platforms can provide personalized learning experiences, catering to individual student needs. AI tutors can provide customized feedback and support, while AI-powered assessment tools can track student progress and identify areas for improvement.

## 5. Challenges: Ethical Considerations and Workforce Transformation

### 5.1 Ethical Challenges

Artificial intelligence (AI) presents numerous ethical challenges as its capabilities expand. One major concern is bias in AI systems, which can perpetuate and amplify existing societal inequalities in areas like hiring, criminal justice, and loan applications. Algorithmic bias arises from biased training data or flawed algorithms, leading to discriminatory outcomes. Ensuring fairness and transparency in AI requires careful attention to data collection, algorithm design, and ongoing monitoring. Another key issue is the potential for job displacement due to AI-driven automation, necessitating workforce retraining and adaptation strategies.

Looking ahead, the ethical implications of AI become even more complex. The development of autonomous weapons systems (AWS) raises concerns about accountability and the potential for unintended consequences in warfare. The increasing sophistication of AI-generated content, including deepfakes, poses a threat to truth and trust in information. Addressing these challenges requires a multi-faceted approach involving collaboration between researchers, policymakers, and the public. Establishing clear ethical guidelines, regulations, and international agreements is essential to harness the benefits of AI while mitigating its risks and ensuring a future where AI aligns with human values.

### 5.2 Workforce Transformation

The integration of artificial intelligence (AI) is expected to cause significant shifts in the job market. Automation driven by AI could displace workers in roles involving repetitive tasks, data processing, and even some cognitive functions. However, AI is also creating new job opportunities in fields such as AI development, data science, AI maintenance, and AI training. The net effect on employment is still debated, with some studies predicting overall job losses and others anticipating a net increase in jobs due to AI-driven economic growth and innovation.

To adapt to these changes, the workforce will need to acquire new skills. Demand is expected to rise for roles requiring uniquely human skills, such as creativity, critical thinking, emotional intelligence, and complex problem-solving. Education and training programs will need to evolve to equip workers with these skills, as well as provide opportunities for reskilling and upskilling. Governments and organizations are exploring policies and initiatives to support workforce transitions, including investments in education, training programs, and social safety nets. Continuous learning and adaptation will be crucial for workers to thrive in the AI-driven economy.

## 6. Future Directions: Towards AGI and Beyond

The ultimate goal of AI research is often considered to be the development of Artificial General Intelligence (AGI), a hypothetical AI that can understand, learn, and perform any intellectual task that a human being can. Predictions for the arrival of AGI vary widely among experts. Some researchers believe AGI is decades or even centuries away, citing the current limitations of AI in areas such as common sense reasoning, understanding context, and adapting to new situations. They argue that significant breakthroughs in AI theory and architecture are needed before AGI can be realized.

Other experts are more optimistic, suggesting AGI could emerge within the next few decades, potentially by the mid-21st century. These predictions often assume that current trends in AI, such as deep learning and neural networks, will continue to advance exponentially. Additionally, some futurists propose that AGI could arise unexpectedly as AI systems become more complex and capable of self-improvement. However, it's important to note that these timelines are speculative, and there is no consensus on when or even if AGI will be achieved. The development of AGI remains one of the greatest challenges and most debated topics in the field of artificial intelligence.

Beyond AGI, future directions in AI research include:

*   **Explainable AI (XAI):** Developing AI systems that can explain their decisions and reasoning processes, making them more transparent and trustworthy.
*   **Federated Learning:** Enabling AI models to be trained on decentralized data sources, protecting data privacy and security.
*   **Neuromorphic Computing:** Designing AI hardware that mimics the structure and function of the human brain, leading to more efficient and powerful AI systems.
*   **AI for Social Good:** Applying AI to address global challenges such as climate change, poverty, and disease.

## 7. Conclusion

The future of AI is bright, with the potential to transform industries, improve lives, and address some of the world's most pressing challenges. However, realizing this potential requires careful consideration of the ethical implications of AI, proactive efforts to mitigate the risks of job displacement, and continued investment in research and development. By fostering collaboration between researchers, policymakers, and the public, we can ensure that AI is developed and deployed in a responsible and beneficial manner, creating a future where AI aligns with human values and contributes to a more prosperous and equitable world.



📌 Short Summary: This report analyzes the future of AI, covering trends like democratization, ethical AI, NLP, and computer vision. It explores AI applications in healthcare, transportation, and manufacturing, while also addressing challenges such as ethical concerns and workforce transformation. The report discusses the prospect of AGI and concludes by emphasizing the need for responsible AI development and collaboration.

📌 Follow-up Questions: ['What specific policies can governments implement to mitigate the negative impacts of AI-driven job displacement and ensure a just transition for workers?', 'How can we develop robust evaluation metrics to assess the fairness and transparency of AI algorithms, and what mechanisms can be put in place to rectify biases?', 'What are the most promising approaches for integrating AI with other emerging technologies, such as blockchain and edge computing, to create novel applications and solutions?', 'How can international cooperation be fostered 